Importações que seram usadas ao longo do projeto

In [ ]:
import kaggle
import pandas as pd
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder,MinMaxScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from yellowbrick.classifier import ConfusionMatrix
import warnings
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.stats.multicomp import MultiComparison
warnings.filterwarnings('ignore')

Utilização do kaggle API para fazer dowload da base de dados a ser usada

In [ ]:
kaggle.api.dataset_download_files('blastchar/telco-customer-churn', path='data', unzip=True)

In [ ]:
data_set=pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

Visualização da base de dados

In [ ]:
data_set

Fiz a remoção do ID dos indivíduos pois não são relevantes para a predição

In [ ]:
data_set.drop(columns=['customerID'],axis=1,inplace=True)

Depois de visualizar as colunas e informações que a base de dados passa , vamos agora aprofundar mais

In [ ]:
data_set.info()

In [ ]:
data_set.describe()

Visualizar se há valores nulos

In [ ]:
data_set.isnull().sum()

Podemos ver que não há valores nulos , logo podemos ir adiante

A coluna TotalChagers deveria ser do valor float mas está em object , logo mudarei para float

In [ ]:
data_set.loc[data_set['TotalCharges']==' ','TotalCharges']='1000000000'

In [ ]:
data_set['TotalCharges']=data_set['TotalCharges'].astype(float)

In [ ]:
media=data_set.loc[data_set['TotalCharges']!=1000000000,'TotalCharges'].mean()

In [ ]:
data_set.loc[data_set['TotalCharges']==1000000000,'TotalCharges']=media

In [ ]:
data_set['TotalCharges'].mean()

O tratamento sobre Total Charges agora está feito

In [ ]:
data_set

In [ ]:
data_set.info()

In [ ]:
for i,column in zip(range(0,len(data_set.columns)),data_set.columns):
    print(i,data_set[column].unique())

In [ ]:
fig,axis=plt.subplots(2,2,figsize=(10,8))
sns.countplot(ax=axis[0][0],x=data_set.iloc[:,1],hue=data_set['Churn'])
sns.countplot(ax=axis[0][1],x=data_set.iloc[:,2],hue=data_set['Churn'])
sns.countplot(ax=axis[1][0],x=data_set.iloc[:,3],hue=data_set['Churn'])
sns.countplot(ax=axis[1][1],x=data_set.iloc[:,5],hue=data_set['Churn'])
fig.show()

In [ ]:
fig,axis=plt.subplots(2,2,figsize=(10,8))
sns.countplot(ax=axis[0][0],x=data_set.iloc[:,6],hue=data_set['Churn'])
sns.countplot(ax=axis[0][1],x=data_set.iloc[:,7],hue=data_set['Churn'])
sns.countplot(ax=axis[1][0],x=data_set.iloc[:,8],hue=data_set['Churn'])
sns.countplot(ax=axis[1][1],x=data_set.iloc[:,9],hue=data_set['Churn'])
fig.show()

In [ ]:
fig,axis=plt.subplots(2,2,figsize=(10,8))
sns.countplot(ax=axis[0][0],x=data_set.iloc[:,10],hue=data_set['Churn'])
sns.countplot(ax=axis[0][1],x=data_set.iloc[:,11],hue=data_set['Churn'])
sns.countplot(ax=axis[1][0],x=data_set.iloc[:,12],hue=data_set['Churn'])
sns.countplot(ax=axis[1][1],x=data_set.iloc[:,13],hue=data_set['Churn'])
fig.show()

In [ ]:
fig,axis=plt.subplots(1,2,figsize=(10,8))
sns.countplot(ax=axis[0],x=data_set.iloc[:,14],hue=data_set['Churn'])
sns.countplot(ax=axis[1],x=data_set.iloc[:,15],hue=data_set['Churn'])
fig.show()

In [ ]:
fig=plt.figure(figsize=(10,8))
sns.countplot(x=data_set.iloc[:,16],hue=data_set['Churn'])
fig.show()

Pelos gráficos é visto que não há uma distribuição tão desigual que vale a pena tratar algo em relação a essas colunas , por exemplo se alguma delas tivesse 2 ou 3 valores unicos mas 1 só deles sobressaisse 99% em relação aos outros dois poderia ser uma opção retirar essa coluna

Além disso esses gráficos são de muita importâncias pois vemos as categorias que mais possuem churn dentro de uma variável categórica , percebendo que há serviços, formas de pagamentos , tempo de contrato e outros fatores que possuem uma categoria que sobressaem o churn em relação as outras, assim a empresa poderia focar nessas variáveis para evitar que haja tanto churn

Agora visualizaremos os dados numericos

In [ ]:
px.box(data_set,y='tenure')

In [ ]:
px.box(data_set,y='MonthlyCharges')

In [ ]:
px.box(data_set,y='TotalCharges')

In [ ]:
px.scatter(data_set,y='tenure')

In [ ]:
px.scatter(data_set,y='MonthlyCharges')

In [ ]:
px.scatter(data_set,y='TotalCharges')

In [ ]:
px.histogram(data_set,x='TotalCharges')

In [ ]:
fig=plt.figure(figsize=(10,8))
sns.countplot(data_set,x='Churn')
fig.show()

Não existem valores muito exorbitantes que possam ser considerados outliers , logo acaba que não há necessidade de tratar outliers nessa base de dados , e acabou que 
utilizaremos ela somente sem o ID 

Para essa base de dados utilizaremos undersampling e oversampling(Misturando os dois é uma técnica chamada Smoteen) para comparar com o teste utilizando a base original, pois pelo gráfico podemos ver que há uma classe majoritária que é a 'No' logo os modelos podem ficar muito bons em prever valor para No e ficar ruins para Yes. 

In [ ]:
X=data_set.iloc[:,:19].values
X

In [ ]:
Y=data_set['Churn'].values
Y

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
X_test[:,0]

Utilizamos agora um ColumnTransforme par aplicar Standard Scaler e OneHotEncoder em certas colunas da base, já que o one hot vai eliminar a chance do modelo considerar que valores maiores em colunas categóricas teriam mais peso do que valores menores , e aplicamos StandarScaler para escalonar os dados pois muitos modelos trabalham melhores em dados escalonados com média 0 e variância 1

In [ ]:
column_transformer=ColumnTransformer(transformers=[('Onehot',OneHotEncoder(),[6,7,8,9,10,11,12,13,14,16]),('Scaler',StandardScaler(),[1,4,17,18])],remainder='passthrough')

In [ ]:
label0=LabelEncoder()
label2=LabelEncoder()
label3=LabelEncoder()
label5=LabelEncoder()
label15=LabelEncoder()

In [ ]:
X_train[:,0]=label0.fit_transform(X_train[:,0])
X_train[:,2]=label2.fit_transform(X_train[:,2])
X_train[:,3]=label3.fit_transform(X_train[:,3])
X_train[:,5]=label5.fit_transform(X_train[:,5])
X_train[:,15]=label15.fit_transform(X_train[:,15])
X_test[:,0]=label0.transform(X_test[:,0])
X_test[:,2]=label2.transform(X_test[:,2])
X_test[:,3]=label3.transform(X_test[:,3])
X_test[:,5]=label5.transform(X_test[:,5])
X_test[:,15]=label15.transform(X_test[:,15])

In [ ]:
X_train=column_transformer.fit_transform(X_train)
X_test=column_transformer.transform(X_test)

In [ ]:
label=LabelEncoder()

In [ ]:
Y_train=label.fit_transform(Y_train)
Y_test=label.transform(Y_test)

In [ ]:
sm=SMOTEENN(random_state=0)

In [ ]:
X_aux=X
Y_aux=Y

In [ ]:
X_aux[:,0]=label0.fit_transform(X_aux[:,0])
X_aux[:,2]=label2.fit_transform(X_aux[:,2])
X_aux[:,3]=label3.fit_transform(X_aux[:,3])
X_aux[:,5]=label5.fit_transform(X_aux[:,5])
X_aux[:,15]=label15.fit_transform(X_aux[:,15])

In [ ]:
X_aux=column_transformer.transform(X_aux)
Y_aux=label.transform(Y_aux)

In [ ]:
X_train_smoteen,Y_train_smoteen=sm.fit_resample(X_aux,Y_aux)

In [ ]:
X_train_over,X_test_over,Y_train_over,Y_test_over=train_test_split(X_train_smoteen,Y_train_smoteen,test_size=0.2,random_state=0)

In [ ]:
fig=plt.figure(figsize=(10,8))
sns.countplot(x=Y_train)
fig.show()

In [ ]:
fig=plt.figure(figsize=(10,8))
sns.countplot(x=Y_train_over)
fig.show()

Treino e teste divididos tanto com base original e Smoteen

Definindo os modelos a serem usados e suas parametrizações para serem testada pelo GridSearch

In [ ]:
modelos=[('RandomForest',RandomForestClassifier(random_state=42),{'model__n_estimators':[200,220,240],'model__max_depth':[10],'model__min_samples_split':[2,4,6],'model__min_samples_leaf':[1,3,5],'model__criterion':['gini','entropy','log_loss']}),
        ('KNN',KNeighborsClassifier(),{'model__n_neighbors': [3, 5, 7],'model__weights': ['uniform', 'distance']}),
        ('GradientBoosting',GradientBoostingClassifier(random_state=42),{'model__n_estimators': [200],'model__learning_rate': [0.01, 0.1]}),
        ('DecisionTree',DecisionTreeClassifier(),{'model__criterion':['gini','entropy','log_loss'],'model__max_depth':[None, 10, 20],'model__splitter':['best','random']}),
        ('SVC',SVC(random_state=0,class_weight='balanced'),{'model__C': [1,5,10],'model__gamma': ['scale', 'auto']}),
        ('LogisticRegressor',LogisticRegression(random_state=42, class_weight='balanced'),{'model__C': [1,5,10],'model__penalty': ['l1','l2']}),
        ('Ada Boost', AdaBoostClassifier(random_state=42),{'model__n_estimators': [100, 150, 200],'model__learning_rate': [0.05, 0.1, 0.5]}),  
    ('XG Boost', XGBClassifier(random_state=42),
        {'model__n_estimators': [100, 150, 200],
         'model__learning_rate': [0.05, 0.1, 0.5]})
        ]

Fazendo o teste de parametrização com GridSearch e armazenando accuracy para cada modelo e as melhores parametrizações, como também o melhor modelo

In [ ]:
def Grid_search_test(modelos,X,Y):
    melhores_params_models=[]
    for name,model,params in modelos:
        pipeline=Pipeline([('model',model)])
        grid=GridSearchCV(estimator=pipeline,param_grid=params,verbose=2,n_jobs=-1)
        grid.fit(X,Y)
        pipeline=grid.best_estimator_
        melhores_params_models.append({'Name':name,'Best Estimator':pipeline})
    return melhores_params_models
best_models_params=Grid_search_test(modelos,X_train,Y_train)
best_models_params_smoteen=Grid_search_test(modelos,X_train_over,Y_train_over)


In [ ]:
def test_models(best_models,X_test,Y_test):
    models_scores=[]
    for modelo in best_models:
        previsao=modelo['Best Estimator'].predict(X_test)
        accuracy=accuracy_score(Y_test,previsao)
        models_scores.append({'Accuracy':accuracy,'Name':modelo['Name']})
    return models_scores
model_score=test_models(best_models_params,X_test,Y_test)
model_score_smoteen=test_models(best_models_params_smoteen,X_test_over,Y_test_over)

In [ ]:
def plot(model_scores):
    fig=plt.figure(figsize=(10,5))
    ax=sns.barplot(data=pd.DataFrame(model_scores),hue='Name',y='Accuracy',palette='pastel',legend=True)
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2, p.get_height() + 0.01, 
                f"{p.get_height():.2%}", ha='center')
    plt.title("Acurácia dos modelos")
    plt.ylabel("Acurácia")
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()

In [ ]:
plot(model_score)

In [ ]:
plot(model_score_smoteen)

Os model scores com smoteen geraram resultados muito melhores , logo mandamos os modelos para a pasta model com joblib dump

In [ ]:

for modelo in best_models_params_smoteen:
    joblib.dump(modelo['Best Estimator'],f"models/{modelo['Name']}")

Com os modelos todos na pasta models podemos então dar load neles para fazer o KFold entre eles

In [ ]:
resultados_KNN=[]
resultados_Logisticregressor=[]
resultados_Decisiontree=[]
resultados_Randomforest=[]
resultados_SVC=[]
resultados_GradientBoosting=[]
resultados_AdaBoost=[]
resultados_XGBoost=[]
for i in range(30):
    k_fold=KFold(n_splits=10,random_state=i,shuffle=True)
    knn=joblib.load('models/KNN')
    lor=joblib.load('models/LogisticRegressor')
    dectree=joblib.load('models/DecisionTree')
    ranfor=joblib.load('models/RandomForest')
    svc=joblib.load('models/SVC')
    gb=joblib.load('models/GradientBoosting')
    ada=joblib.load('models/Ada Boost')
    xgb=joblib.load('models/XG Boost')
    score_knn=cross_val_score(knn,X_train_over,Y_train_over,cv=k_fold)
    score_lor=cross_val_score(lor,X_train_over,Y_train_over,cv=k_fold)
    score_dectree=cross_val_score(dectree,X_train_over,Y_train_over,cv=k_fold)
    score_ranfor=cross_val_score(ranfor,X_train_over,Y_train_over,cv=k_fold)
    score_svc=cross_val_score(svc,X_train_over,Y_train_over,cv=k_fold)
    score_gb=cross_val_score(gb,X_train_over,Y_train_over,cv=k_fold)
    score_ada=cross_val_score(ada,X_train_over,Y_train_over,cv=k_fold)
    score_xgb=cross_val_score(xgb,X_train_over,Y_train_over,cv=k_fold)
    resultados_KNN.append(score_knn.mean())
    resultados_Logisticregressor.append(score_lor.mean())
    resultados_Decisiontree.append(score_dectree.mean())
    resultados_Randomforest.append(score_ranfor.mean())
    resultados_SVC.append(score_svc.mean())
    resultados_GradientBoosting.append(score_gb.mean())
    resultados_AdaBoost.append(score_ada.mean())
    resultados_XGBoost.append(score_xgb.mean())
    print(i)

In [ ]:
lista=['KNN','Logistic','Decision','Random','SVC','Gradient','Ada','XGBoost']

In [ ]:
resultados_concatenados=[resultados_KNN,resultados_Logisticregressor,resultados_Decisiontree,resultados_Randomforest,resultados_SVC,resultados_GradientBoosting,resultados_AdaBoost,resultados_XGBoost]
resultados_dict=[]

In [ ]:
cont=0
pos=0
for i in range(0,240):
    if pos==30:
        pos=0
    if i%30==0 and i!=0:
        cont+=1
    resultados_dict.append({'Accuracy':resultados_concatenados[cont][pos],'Modelo':lista[cont]})
    pos+=1
    

In [ ]:
data_resultados=pd.DataFrame(data=resultados_dict)

In [ ]:
multi_comparison=MultiComparison(data_resultados['Accuracy'],data_resultados['Modelo'])

Aqui pegamos os modelos e os colocamos na função de plot simultaneous do MultiComparison utilizando Tukeyhsd

In [ ]:
multi_comparison.tukeyhsd().plot_simultaneous()

Com os modelos ja carregados podemos ver os gráficos sem ter de rodar o GridSearch

In [ ]:
knn=joblib.load('models/KNN')
lor=joblib.load('models/LogisticRegressor')
dectree=joblib.load('models/DecisionTree')
ranfor=joblib.load('models/RandomForest')
svc=joblib.load('models/SVC')
gb=joblib.load('models/GradientBoosting')
ada=joblib.load('models/Ada Boost')
xgb=joblib.load('models/XG Boost')

In [ ]:
lista=['Random Forest','KNN','GradientBoost','Decision Tree','SVC','Logistic Regressor','Ada Boost','XGBoost']

In [ ]:
modelos_para_fit=[ranfor,knn,gb,dectree,svc,lor,ada,xgb]

In [ ]:
model_score_sem_grid=[]
model_score_sem_grid_smoteen=[]
def modelo_accuracy(modelo,nome,X,Y):
    previsao=modelo.predict(X)
    accuracy=accuracy_score(Y,previsao)
    return {'Name':nome,'Accuracy':accuracy}
for i in range(0,len(modelos_para_fit)):
    model_score_sem_grid.append(modelo_accuracy(modelos_para_fit[i],lista[i],X_test,Y_test))
    model_score_sem_grid_smoteen.append(modelo_accuracy(modelos_para_fit[i],lista[i],X_test_over,Y_test_over))


In [ ]:
plot(model_score_sem_grid)

In [ ]:
plot(model_score_sem_grid_smoteen)

No final pode ser visto que o melhor modelo é o KNN que ficou a frente dos outros 7 modelos , com média de aproximadamente 96% de accuracy nos teste em KFold,liderando no gráfico do Multicomparison e no gráfico geral dos modelos quando testando sobre a base smoteen

O KNN se sobresaiu pois pelo visto modelos como o SVC não conseguiram se adaptar tanto a base de dados ( na qual o SVC tenta traçar uma reta, plano e outros modelos matemáticos para se adaptar a um problema), e também modelos que utilizam de árvore(Random forest, decision tree ,Gradient Boosting ,Ada e XGBoost ) não geraram uma adaptação boa para meio que traçar uma sequencia de regras para chegar ao target correto, e o Logistic Regressor não conseguiu se adaptar de forma linear ao problema e assim o KNN foi melhor pelo fato dos dados conseguirem ser mais bem adaptados a um modelo que preve atraveś de calculos de distância.

In [ ]:
tabela=[]
for i,model in zip(range(0,len(lista)),modelos_para_fit):
    previsao=model.predict(X_test_over)
    tabela.append({'Nome':lista[i],'Accuracy':accuracy_score(Y_test_over,previsao),'MAE':mean_absolute_error(Y_test_over,previsao),'MSE':mean_squared_error(Y_test_over,previsao),'RMSE':pow(mean_squared_error(Y_test_over,previsao),0.5)})


In [ ]:
tabela_df=pd.DataFrame(data=tabela)

In [ ]:
tabela_df

Como é visto pela tabela o KNN merece o lugar de ser o melhor algoritmo para prever Churn com essa base de dados com Smoteen,que balanceia os dados quanto ao target de forma que aplica oversampling e retira targets que são de uma classe mas estão muito próximos de outros de outra classe. 

In [ ]:
knn=best_models_params[1]['Best Estimator']

In [ ]:
knn

In [ ]:
cm=ConfusionMatrix(knn)
cm.fit(X_train,Y_train)
cm.score(X_test,Y_test)

In [ ]:
cm=ConfusionMatrix(knn)
cm.fit(X_train,Y_train)
cm.score(X_test_over,Y_test_over)

In [ ]:
knn_smoteen=best_models_params_smoteen[1]['Best Estimator']

In [ ]:
knn_smoteen

In [ ]:
cm_smoteen=ConfusionMatrix(knn_smoteen)
cm_smoteen.fit(X_train_over,Y_train_over)
cm_smoteen.score(X_test_over,Y_test_over)

In [ ]:
cm_smoteen=ConfusionMatrix(knn_smoteen)
cm_smoteen.fit(X_train_over,Y_train_over)
cm_smoteen.score(X_test,Y_test)

In [ ]:
print(classification_report(Y_test,knn.predict(X_test)))

In [ ]:
print(classification_report(Y_test,knn_smoteen.predict(X_test)))

Como devemos comparar para mesmos teste pegamos então o knn que foi feito o fit com X e Y normais e outro knn que foi feito o fit com X e Y over, logo testando eles sobre Y_test vemos que no caso de predição de churn o modelo treinado com smoteen obteve resultado melhores para identificar a classe 1 já que os registros que eram 1 ele acertou 93%(recall)  , mesmo que dos que ele classificou como 1 ter acertado somente 48%(precisão) ele ainda obteve um modelo bem melhor do que o treinado com X_test e Y_test pois para churn obteve 54% que é 39% a menos do modelo com smoteen e precisão só 7% a mais do que o smoteen , assim não compensando o ganho de precisão visto que o recall é muito menor para a classe que é o objetivo da predição e que daria mais prejuízo se passasse despercebida.